In [28]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats

In [29]:
def getAveragePer1000(filepath,n,filename):
    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    averages = [np.mean(csv_df.loc[i:i+n-1, "P"]) for i in range(0, csv_df.shape[0], n)]
    np.savetxt('../../similar_region_average1000/'+filename+'.csv', averages, delimiter = ',')
    return averages

In [30]:
def ifExistCSVFile_1000Average(filename):
    filepath='../../similar_region_average1000/'
    files=os.listdir(filepath)
    if filename in files:
        return True
    else:
        return False

In [31]:
arrforDistribution=[]
for curDir, dirs, files in os.walk(top="../../similar_region_csvfiles/"):
    for file in files:
        if file.startswith("pos_prob_split_"):
            fileprefix=file[file.index("chr"):file.index(".")]
            filepath=os.path.join(curDir,file)
            arrforDistribution+=getAveragePer1000(filepath=filepath,n=1000,filename=fileprefix)

In [32]:
mhc_1000=getAveragePer1000(filename="mhc",filepath="../../similar_region_csvfiles/posmhc.csv",n=1000)
arrforDistribution+=mhc_1000

In [33]:
#get mean of Distribution
mean_arr=np.mean(arrforDistribution)
#get standard deviation of Distribution
std_arr=np.std(arrforDistribution)
print("mean_arr",mean_arr)
print("std_arr",std_arr)

mean_arr -0.017462454026226405
std_arr 0.006996042091888453


In [34]:
def compareWith0_01Pvalue(mean_arr,std_arr,filename,filepath):
    arr=[]
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    for i in range(len(csv_matrix)):
        mid_res=stats.norm.cdf(csv_matrix[i], mean_arr, std_arr)
        if mid_res<0.05:
            arr.append(1)
        else:
            arr.append(0)
    np.savetxt("../../similar_region_compare_with_01/"+filename+".csv", arr, delimiter = ',')
    return filename

In [35]:
for curDir, dirs, files in os.walk(top="../../similar_region_average1000/"):
    for file in files:            
        filename=file[:file.index(".")]
        prefix=filename.split("_")[0]
        filepath=os.path.join(curDir,file)
        #print(filepath)
        compareWith0_01Pvalue(mean_arr=mean_arr,std_arr=std_arr,filename=filename,filepath=filepath)

In [36]:
def generatedfandcsv(columns,dic,filename):

    row_names=list(dic.keys())
    a=[]
    bs=[]
    #cs=[]
    for x in row_names:
        a.append(x)
        bs.append(dic[x])
        #cs.append(dic[x][1])
    dictforDF=dict()
    dictforDF[columns[0]]=a
    dictforDF[columns[1]]=bs
    #dictforDF[columns[2]]=cs

    df_index=pd.DataFrame(dictforDF)
    df_index.to_csv("../../similar_region_csvfile_start_length/"+filename+"_start_lenth.csv")
    return df_index


In [37]:
def checkContinuous1(arr):
    res=dict()
    i=0
    while i<len(arr)-1:

        if arr[i]==1:

            count=1
            for j in range(i+1,len(arr),1):
                if arr[j]==1:
                    count+=1
                    
                else:
                    
                    break

            res[i]=count
            i+=count

        else:
            i+=1
            continue
    return res
            


In [38]:
def getstartAndLength(filepath,filename):
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    res=checkContinuous1(csv_matrix)
    df=generatedfandcsv(columns=["start","length"],dic=res,filename=filename)
    return df

In [39]:
for curDir, dirs, files in os.walk(top="../../similar_region_compare_with_01/"):
    for file in files:
        filename=file[:file.index(".")]
        filepath=os.path.join(curDir,file)
        getstartAndLength(filepath=filepath,filename=filename)

In [40]:

def countNumofLength(dic,filepath):
    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr=np.array(csv_df['length'])
    #print(dic)
    for i in range(len(arr)):
        #print("before iteration",dic)
        if arr[i] in dic:
            #print("in iteration not",dic)
            dic[arr[i]]+=1
        else:
            dic[arr[i]]=1
            #print("in iteration",dic)
    #print(dic)
    return dic

In [41]:

mydic=dict()
mhcdic=dict()
for curDir, dirs, files in os.walk(top="../../similar_region_csvfile_start_length/"):
    for file in files:
            filename=file[:file.index(".")]
            if filename!="mhc_start_lenth":                  
                filepath=os.path.join(curDir,file)
                countNumofLength(mydic,filepath=filepath)
            else:
                 filepath_mhc=os.path.join(curDir,file)
                 countNumofLength(mhcdic,filepath=filepath_mhc)

In [42]:
mhcdic

{16: 1, 2: 2, 1: 1, 3: 1, 21: 1, 28: 1, 5: 1}

In [43]:
mydic

{1: 2041, 2: 425, 4: 32, 3: 118, 7: 2, 5: 11, 6: 5}

In [44]:
arr1=[]

for k2,arr in mydic.items():
    for i in range(arr):
        arr1.append(k2)

for k2,arr in mhcdic.items():
    for i in range(arr):
        arr1.append(k2)

In [45]:
mean_1=np.mean(arr1)
dev1=np.std(arr1)
print("mean",mean_1)
print("dev",dev1)
mid_res=stats.norm.ppf(0.9999999999,mean_1,dev1)
mid_res

mean 1.3436790310370932
dev 0.9961962935046483


7.680823247073226

In [46]:

def getAllPositions(filepath):
    pos=[]
    #vcf_in=vcf(filepath)
    vcf_file = pysam.VariantFile(filepath)
    for record in vcf_file:

        pos.append(record.pos)
    pos=sorted(list(set(pos)))
    return pos

In [47]:
def findPosition(filepath,start,length):
    arr=[]
    allpos=getAllPositions(filepath=filepath)
    arr.append(allpos[start*1000])
    if (start+length)*1000-1<len(allpos):
        end=allpos[(start+length)*1000-1]
    else:
        end=allpos[-1]
    arr.append(end)
    return arr



In [48]:
# filepath="../../similar_region_csvfile_start_length/chr1_xab_start_lenth.csv"


In [49]:
# csv_data = pd.read_csv(filepath, low_memory = False)
# csv_df = pd.DataFrame(csv_data)
# arr_start=np.array(csv_df['start'])
# arr_length=np.array(csv_df['length'])

# mydic=dict()
# midarr=[]
# for i in range(len(arr_start)):
#     mydic[arr_start[i]]=arr_length[i]
# mydic

In [50]:

def GetPosition(filepath,res,threshold):
    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]
    filename=(filepath.split("/")[-1]).split("_start_lenth.csv")[0]
    findfoldername="split_"+filename.split("_")[0]
    findfoldername_filename=filename.split("_")[1].split(".")[0]
    filepath="../../"+findfoldername+"/"+findfoldername_filename+".vcf.gz"
    for k,v in mydic.items():
        if v>=threshold:
            print("filepath",filepath)
            midarr.append(findPosition(filepath=filepath,start=k,length=v))
            
        else:
            continue
    if midarr!=[]:
        res[findfoldername+"_"+findfoldername_filename]=midarr
    return res

In [51]:
def getPositionforMHC(res):
    mhc_start_length_path="/data2/wangxuedong/mhc_test_data/similar_region_csvfile_start_length/mhc_start_lenth.csv"
    csv_data = pd.read_csv(mhc_start_length_path, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]
    for k,v in mydic.items():
        if v>=8:
            midarr.append(findPosition(filepath="../../mhcdataset/1kgp.29720000-33130000.vcf",start=k,length=v))            
        else:
            continue
    res["MHC"]=midarr
    return res
            



In [52]:
res=dict()
mhcres=dict()
for curDir, dirs, files in os.walk(top="../../similar_region_csvfile_start_length/"):
    for file in files:
            filename=file[:file.index(".")]
            if filename!="mhc_start_lenth":
                filepath=os.path.join(curDir,file)
                GetPosition(filepath=filepath,res=res,threshold=8)
            else:
                getPositionforMHC(res=mhcres)

In [53]:
res

{}

In [54]:
mhcres

{'MHC': [[29720403, 30048796], [30994370, 31528792], [32212726, 32882258]]}